# Leader Election
* Bank account details are replicated at a few servers, but one of these servers are responsible for all reads and writes, that is leader among replicas.
    * What is leader crash?
    * what if there are more than 1 leader?
    * WHat if servers disagree for leader?
* In the sequencer based algorithm for total ordering  of multicast, the sequencer is a leader
* Group of NTP servers, who is root servers.
* Apache zookeeper, google's chubby
* From group of process we have to choose leader and let everyone know.
* N process, each process has unique id like ip address and port number, Failure may occur during election.

### Calling for election
* ANy process can cal for an election
* A process can call for at most one election at a time
* Multiple process can call for election. All of them together yield single leader.
* Result of election should not depend on which process calls for it.
* At the end of election protocol the non faulty process with the best election attribute value is elected. Common attribute, leader with highest id. or fasted cpu or most disk space, or most number of files.

### Ring leader election
* Nprocess are in logical ring, ith process has communication channel to (i+1) % N.
* All message are sent clockwise around the ring.
![](images/ring.jpg)

* Any process Pi, which discovers the old coordinator has failed initiate an Election message with its own id:attr.
* When other process receive this message, it compares it value with own attributes.
* If attribute is greater, then forward the message
* If smaller then overwrite with own id:attr and forwards it.
* After making one circle message again come to same process ,then that process send Elected message to neighbor with its id.
* ALl process mark their elected variable = elected process id
* 3N - 1 message in worst case. Best case then 2N messages.
* Multiple initiator
    - Each process cache initiator of election.
    - Each process suppresses election message of any lower id initiators.
    - Update cache if receive higher id initiator's message
* Effect of failure:
    - Elected node failed then, predecessor detect failure then start new election.

###  Google'e Chubby
* System for locking
* Group of replicas, need to have a master server elected at all time.
* Potential leader tries to get votes from other servers
* Each server votes for at most one leader
* Server with majority of votes become new leader and inform everyone.
* After election finishes other servers promise not to run election again for a while. While = master lease time = few seconds
* Master lease can be renewed by the master as long as it continues to win a majority each time.
* If master fails, automatic re election will conducted.

### Zookeeper
* Each server creates new sequence number (ids). Elect the highest id server as leader.
* Everyone monitors current master, on failure re election, it can lead to flood of election messages
* Each process monitors its next higher id process.
* If that successor was the leader and it has failed, become the new leader.
* What if id conflicts?
    - 2 phase commit, Leader sends NEW_lEADER message to all. Each process responds with ACK to at most 1 leader that is one with highest process id. Leader waits for a majority of ACKS and then sends commit to all. On commit everyone update leader variable.
    
### Bully algorithm
* All process know other process id.
* When a process finds the coordinator has failed via failure detector,
    - If it knows it is highest id, it elects it self as coordinator and send message to everyone with lower id, that election is completed. 
    - Else initiates an election by sending Election message
        - Election message is sent to process with higher id.
        - If receives no answer within timeout, calls itself leader and sends message to all lower id process that election completed I am leader
        - If answer receive from high id process, wait for coordinator message, if not receive within timeout start new election.

# Mutual Exclusion
* Bank's server in cloud: simultaneously 2 customer put money in my account. Initially balance was 1000. BOth customer trying to add 10000. Both read initial value concurrently from bank's server. Both add 10000 to this amount and write final amount to server.
* To resolve it we can provide mutual exclusive access to account, allow only 1 client to access account.
* In Distributed system locking  files is needed 
* Accessing objects in safe and consistent way
* Chubby is google's locking system
* Apache zookeeper is used to coordinate among server.
* Critical section problem
    - Piece of code for which we need to ensure there is at most one process is executing it at any given point of time.
    - enter() to enter in critical section
    - AccessResource() to run critical section code
    - exit() to exit critical section.
* For single OS structure,
    - We can use Semaphores, mutexes, condition variables, monitors.
    - Semaphore is an integer that can be accessed via 2 special function
    - Semaphore S = 1// mac allowed process
    - wait(S) before entering the section and signal(S) to exit the section.
* In distributed system shared variable approach is not possible.

### Distributed system Central algorithm
* Elect central master/leader using election algorithm
* Master keeps queue of waiting process, who wish to access the CS.
* A special token which allows its holder to access CS.
* Action of any process in a group
    - enter(): send request to master, wait for token from master
    - exit() : Send back token to master

* 2 message for each enter, 1 message for each exit
* Master is single point of failure.

### Ring based mutual exclusion
![](images/ring_mutual.jpg)

* N process organized in virtual ring.
* Each process can send  message to its successor in ring
* Exactly 1 token
* enter(): wait until you get token
* exit(): pass token to successor
* If receive token and currently not in enter(), pass to successor.
* Per request message might be as high as N
* 1 message per exit()

### Ricart-Agrawala'a problem
*  When process Pi want to enter CS it call enter(), it multicast message to all process, request <T,Pi> where T = current Lamport timestamp at Pi
* Wait until all other process have responded positively to request. it change state to Held.
* Request are granted in order of causality
* Pi in request <T, Pi> is used to break ties, as lamport timestamp are not unique for concurrent events.
* Suppose process Pi receive request of <Tj, Pj>
    - if Pi's state is Held or wanted and (Ti, i) < (Tj, j) add request to local queue
    - else reply to Pj
* exit() at Pi
    - change state to released and send reply to all queued process

### Maekawa's algorithms
* Above algo require response from all the process in a group.
* Instead we can get reply from some of the process.
* Each process Pi asscoated with voting set Vi
* Each process belongs to its own voting set.
* Intersection of any 2 voting state must not be empty
* Each voting set has size K. Each process belongs to M other voting set, K = M = $\sqrt{N}$ works best

* one way of doing it is put N process in $\sqrt{N}$ by $\sqrt{N}$ matrix and for each Pi, its voting set Vi = row containing Pi + column containing Pi

![](images/Maekawa.jpg)

* Each process request permission from only its voting set members.
* Each process gives permission to at most 1 process at a time.
* state = released, voted = false
* enter() at process Pi
    - state = wanted
    - Multicast request message to all processes in Vi
    - wait for Reply message from all Vi processes
    - state = held
* exit()
    - state = released
    - Multicast Release to all process in Vi.
* If Pi receive request fro Pj
    - if (state == Held or voted =true)
        - queue request
    - send Reply to Pj and set voted = true
    - When Pi receives a release from Pj
    - if (queue empty)
        - voted = false
    - else
        - dequeue head of queue, say Pk
        - send reply to Pk
        - voted = true.

* only 1 group's one process will win,
    - suppose Pi in 2 group it can send yes to only one of the process of specific group.

* 2$\sqrt{N}$ messages per enter, $\sqrt{N}$ messages per exit

--------------------
--------------------

### RPC
* Remote procedure call
* Allow to call other process' function.
* Other variant in object based model is RMI (Remote Method Invocation)
* Local procedure call LPC
    - Call from 1 function to other function in the same process
    - Use stack to pass arguments and return values
    - Access object via pointer or reference
    - LPC has exactly once semantics. Function gets called EXACTLy once.
* In RPC we can not use pointer, we use global reference like IP + Port + object number.
* RPC can also be in same host, like between two different process in same computer
* Problem is 
    - Request message might be dropped
    - Reply message is dropped
    - Called process fails before or after execution of called function.
    - Request message is duplicated.
![](images/RPC.jpg)

* Idempotent operation can tolerate at least 1, semantics .For example x = 1
* Non idempotent like x+=1 .
![](images/stub.jpg)

* Client stub: same function signature as callee(). Allows same caller() code for LPC and RPC
* Communication module, forwards requests and replies to appropriate hosts server
* Dispatcher: Selects which server stub to forward request to.
* Server stub: Calls callee(), allows it to return value

* Programmer write code only for caller and callee. Code for remaining components generates automatically from function signature.
* Ex. CORBA, Sun RPC, JAVA RMI.

#### Marshalling
* Different architecture use different ways of representing data,
    - Big endian(IBM, system 360), Little endian (Intel)
* Caller and callee process uses its own platform dependent way of storing data, middleware has a common data representation (CDR), platform independent.
* Caller process converts argument to CDR, (Marshalling)
* Callee process extracts argument from message into own platform dependent form (Unmarshalling)
* Return values are again marshalled and unmarshalled.

### Transaction
* Series of operations executed by client
* Each operation is RPC to server
* Transaction either complete and commit at server or aborts and no effect on server.
![](images/rpc_transaction.png)

* Atomicity : all or nothing, a transaction should complete successfully, so its effects are recorded in the server objects or it has no effects at all.
* Isolation: Need transaction to be indivisible (atomic) from other transaction's point of view
    - No access to intermediate result or state of other transaction
    - Free from interference from other transaction.
* ACID property of transaction
    - Atomicity: All or nothing
    - Consistency: If the server starts in the consistent state, the transaction end server in consistent state
    - Isolation: Transaction must be performed without interference from other transaction. Non final effects of transaction must not be visible to other transaction.
    - Durability: After transaction completed successfully, all its effects are saved in permanent storage.
* When we do not care about ACID
![](images/loss_update.JPG)

![](images/Untitled.png)

* To prevent it we can execute transaction one at a time, but we care about number of transaction per seconds for performance and revenue.

### Serial Equivalence
* We have multiple transaction and interleave operation of all transaction in such a way that it output like they have executed serially.
* 2 operations are said to be conflicting operations. if their combined effect depends on the order they are executed.
    * read(x) and write(x)
    * write(x) and read(x)
    * write(x) and write(x)
* 